In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


In [2]:
np.random.seed(42)

n_A = 12000
n_B = 12000

# baseline conversion
p_A = 0.080
p_B = 0.086  # slight uplift

conv_A = np.random.binomial(1, p_A, n_A)
conv_B = np.random.binomial(1, p_B, n_B)

df = pd.DataFrame({
    "variant": ["A"]*n_A + ["B"]*n_B,
    "converted": np.concatenate([conv_A, conv_B])
})

df.head()


,variant,converted
0,A,0
1,A,1
2,A,0
3,A,0
4,A,0


In [3]:
summary = df.groupby("variant")["converted"].agg(["count","sum","mean"])
summary.columns = ["users","conversions","conversion_rate"]
summary


,users,conversions,conversion_rate
variant,,,
A,12000,947,0.078917
B,12000,1083,0.090250


In [4]:
counts = np.array([summary.loc["A","conversions"], summary.loc["B","conversions"]])
nobs = np.array([summary.loc["A","users"], summary.loc["B","users"]])

stat, pval = proportions_ztest(counts, nobs)
stat, pval


(np.float64(-3.1548700977599573), np.float64(0.0016056958077676345))

In [5]:
ci_A = proportion_confint(summary.loc["A","conversions"], summary.loc["A","users"], alpha=0.05, method="wilson")
ci_B = proportion_confint(summary.loc["B","conversions"], summary.loc["B","users"], alpha=0.05, method="wilson")

ci_A, ci_B


((0.07422648891649992, 0.08387635385522939),
 (0.08525352170503919, 0.09550873396656046))

In [6]:
cr_A = summary.loc["A","conversion_rate"]
cr_B = summary.loc["B","conversion_rate"]

uplift_abs = cr_B - cr_A
uplift_rel = uplift_abs / cr_A

uplift_abs, uplift_rel


(np.float64(0.011333333333333334), np.float64(0.1436114044350581))

In [7]:
alpha = 0.05
if pval < alpha and uplift_abs > 0:
    decision = "Ship Variant B (statistically significant uplift)."
else:
    decision = "Do NOT ship Variant B (no statistically significant improvement)."

decision


'Ship Variant B (statistically significant uplift).'

In [ ]:
## Business Question
Does Variant B improve conversion rate compared to Variant A?

## Hypotheses
H0: Conversion rate(B) = Conversion rate(A)  
H1: Conversion rate(B) ≠ Conversion rate(A)

## Success Metric
Primary: Conversion rate


In [ ]:
## Results & Decision

Variant B shows a higher conversion rate compared to Variant A.  
The statistical test confirms that the observed uplift is statistically significant at the 95% confidence level.

## Recommendation

Ship Variant B.  
The improvement in conversion rate is statistically significant and represents a positive business impact with low implementation risk.
